In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import torch
import torchvision
from torch import nn
from d2l import torch as d2l
from tensorflow.keras.models import Sequential
from tensorflow import keras 
import tarfile
import os
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import PIL.ImageEnhance as ImageEnhance
import requests
from concurrent.futures import ThreadPoolExecutor
from sklearn.preprocessing import MinMaxScaler
from torchvision import transforms
from keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, BatchNormalization

ModuleNotFoundError: No module named 'torch'

First setp: Load the data

In [2]:
"""Getting df with all filenames and images (missing labels at this stage)"""

# Path to directory containing image files
image_directory = r'C:\Users\EmiliaGachowetz-Gepp\Documents\New ML\images\flickr_logos_27_dataset_images' #example: unzipped_images_path+ directory_separator +'flickr_logos_27_dataset_images'

# List of all files (images) in the directory
image_filenames = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

# Function to process images and store them in a DataFrame
def process_images(image_directory, image_filenames):
    image_data = []  # List to store image data

    for filename in image_filenames:
        try:
            image_path = os.path.join(image_directory, filename)  # Construct the full path to the image file
            img = Image.open(image_path)  # Open the image using Pillow (PIL)
            img = img.convert('L') #Converting to gray scale
            img_resized = img.resize((100, 100), Image.LANCZOS)  # Resize the image to 100x100 pixels
            # img_array = np.array(img_resized) # Convert image to numpy array if needed
            image_data.append({'Filename': filename, 'Image': img_resized})

        except Exception as e:
            # Handle errors (e.g., if the image cannot be processed)
            print(f"Error processing image {filename}: {e}")

    # Convert the list of dictionaries to a DataFrame
    image_df = pd.DataFrame(image_data)
    return image_df

image_df=process_images(image_directory, image_filenames)

In [3]:
#Display the data frame
image_df

,Filename,Image
0,106523337.jpg,<PIL.Image.Image image mode=L size=100x100 at ...
1,1075391489.jpg,<PIL.Image.Image image mode=L size=100x100 at ...
2,108232382.jpg,<PIL.Image.Image image mode=L size=100x100 at ...
3,108232417.jpg,<PIL.Image.Image image mode=L size=100x100 at ...
4,109950015.jpg,<PIL.Image.Image image mode=L size=100x100 at ...
...,...,...
1074,882748860.jpg,<PIL.Image.Image image mode=L size=100x100 at ...
1075,89201664.jpg,<PIL.Image.Image image mode=L size=100x100 at ...
1076,96668817.jpg,<PIL.Image.Image image mode=L size=100x100 at ...
1077,98498585.jpg,<PIL.Image.Image image mode=L size=100x100 at ...


Second Step: Encode the labels and merge the data such that we have lablled data

In [5]:
#load info about images
images_info=pd.read_csv(r"C:\Users\EmiliaGachowetz-Gepp\Documents\New ML\unzipped\flickr_logos_27_dataset\flickr_logos_27_dataset_training_set_annotation.txt", sep='\s+', header=None)
#renaming the columns correspondingly:
new_column_names = ['Image ID', 'Label', 'Subset', 'x1', 'y1','x2', 'y2']
images_info.columns=new_column_names

images_info
merged_df = pd.merge(image_df, images_info, left_on='Filename', right_on='Image ID', how='inner')
images_full=merged_df.drop('Filename', axis=1)
images_full

,Image,Image ID,Label,Subset,x1,y1,x2,y2
0,<PIL.Image.Image image mode=L size=100x100 at ...,1075391489.jpg,Nbc,6,2,4,77,58
1,<PIL.Image.Image image mode=L size=100x100 at ...,108232382.jpg,Citroen,3,75,29,426,305
2,<PIL.Image.Image image mode=L size=100x100 at ...,108232382.jpg,Citroen,4,75,29,426,305
3,<PIL.Image.Image image mode=L size=100x100 at ...,108232382.jpg,Citroen,5,75,29,426,305
4,<PIL.Image.Image image mode=L size=100x100 at ...,108232382.jpg,Citroen,6,75,29,426,305
...,...,...,...,...,...,...,...,...
4531,<PIL.Image.Image image mode=L size=100x100 at ...,98498585.jpg,Porsche,5,133,72,382,322
4532,<PIL.Image.Image image mode=L size=100x100 at ...,98498585.jpg,Porsche,6,133,72,382,322
4533,<PIL.Image.Image image mode=L size=100x100 at ...,99816106.jpg,Puma,4,164,35,390,246
4534,<PIL.Image.Image image mode=L size=100x100 at ...,99816106.jpg,Puma,5,164,35,390,246


In [6]:
label_encoder = LabelEncoder()
images_full['labels_coded'] = label_encoder.fit_transform(images_full['Label'])
images_full

,Image,Image ID,Label,Subset,x1,y1,x2,y2,labels_coded
0,<PIL.Image.Image image mode=L size=100x100 at ...,1075391489.jpg,Nbc,6,2,4,77,58,15
1,<PIL.Image.Image image mode=L size=100x100 at ...,108232382.jpg,Citroen,3,75,29,426,305,3
2,<PIL.Image.Image image mode=L size=100x100 at ...,108232382.jpg,Citroen,4,75,29,426,305,3
3,<PIL.Image.Image image mode=L size=100x100 at ...,108232382.jpg,Citroen,5,75,29,426,305,3
4,<PIL.Image.Image image mode=L size=100x100 at ...,108232382.jpg,Citroen,6,75,29,426,305,3
...,...,...,...,...,...,...,...,...,...
4531,<PIL.Image.Image image mode=L size=100x100 at ...,98498585.jpg,Porsche,5,133,72,382,322,18
4532,<PIL.Image.Image image mode=L size=100x100 at ...,98498585.jpg,Porsche,6,133,72,382,322,18
4533,<PIL.Image.Image image mode=L size=100x100 at ...,99816106.jpg,Puma,4,164,35,390,246,19
4534,<PIL.Image.Image image mode=L size=100x100 at ...,99816106.jpg,Puma,5,164,35,390,246,19


Third Step: Augment the data

In [7]:
%matplotlib inline

'''This uses torch vision library to augment the data by changing the colour a bit and randonmly flipping the images'''
color_aug = torchvision.transforms.ColorJitter(
    brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)

shape_aug = torchvision.transforms.RandomResizedCrop(
    (100, 100), scale=(0.9, 1), ratio=(0.5, 2))

augs = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(), color_aug, shape_aug])


In [8]:

#def apply_aug(img, aug, num_rows=1, num_cols=2, scale=1.5):
    #Y = [aug(img) for _ in range(num_rows * num_cols)]
    #d2l.show_images(Y, num_rows, num_cols, scale=scale) #ifwe want to plot the augmented images when applying the augmentation

'''This function will apply the above defined augmentation mechanisms'''
def apply_aug(img, aug, filename, label, num_rows=1, num_cols=2):
    augmented_images = [aug(img) for _ in range(num_rows * num_cols)]
    
    # Convert the augmented images to PIL Image objects if needed
    augmented_images = [Image.fromarray(img) if not isinstance(img, Image.Image) else img for img in augmented_images]
    
    # Create a DataFrame to store the augmented images
    #data = {'Filename': [filename] * len(augmented_images), 'Augmented_Image': augmented_images}
    data = {'Filename': [filename] * len(augmented_images), 'Label coded': [label] * len(augmented_images), 'Augmented_Image': augmented_images}
    df = pd.DataFrame(data)
    
    return df

def augment_images(images_df):
    '''This function actually applies the augmentation to images in the DataFrame'''
    dataframes = []  #Creating a list to store the augmented images

    for index, row in images_df.iterrows(): #Loop through each row in the data frame
        label = row['labels_coded']
        filename = row['Image ID']
        
        #Find the original image in the input data frame 
        original_image_row = images_df[images_df['Image ID'] == filename].iloc[0]
        original_image = original_image_row['Image']
        
        #Apply augmentation of the previous function to the original image
        augmented_images_df = apply_aug(original_image, augs, filename, label)
        
        #Append the augmented images to the initilaized list
        dataframes.append(augmented_images_df)

    #transform the list to a new data frame
    augmented_df = pd.concat(dataframes, ignore_index=True)
    
    return augmented_df

In [9]:
#apply augmentation to our data 
augmented_df=augment_images(images_full)

In [55]:
augmented_df

,Filename,Label coded,Augmented_Image
0,1075391489.jpg,15,<PIL.Image.Image image mode=L size=100x100 at ...
1,1075391489.jpg,15,<PIL.Image.Image image mode=L size=100x100 at ...
2,108232382.jpg,3,<PIL.Image.Image image mode=L size=100x100 at ...
3,108232382.jpg,3,<PIL.Image.Image image mode=L size=100x100 at ...
4,108232382.jpg,3,<PIL.Image.Image image mode=L size=100x100 at ...
...,...,...,...
9067,99816106.jpg,19,<PIL.Image.Image image mode=L size=100x100 at ...
9068,99816106.jpg,19,<PIL.Image.Image image mode=L size=100x100 at ...
9069,99816106.jpg,19,<PIL.Image.Image image mode=L size=100x100 at ...
9070,99816106.jpg,19,<PIL.Image.Image image mode=L size=100x100 at ...


Fourth Step: Split into eatures and labels and into training and test set

In [10]:
#split it into features and labels
images = augmented_df.drop(['Filename','Label coded'], axis=1)
labels = augmented_df[['Label coded']]

In [11]:
X_train ,X_test, y_train, y_test=train_test_split(images, labels, test_size=0.2, random_state=42)

In [12]:
# Convert PIL images in DataFrame to NumPy array
X_train = np.array([np.array(img) for img in X_train.iloc[:, 0]])
X_test = np.array([np.array(img) for img in X_test.iloc[:, 0]])

# Reshape the NumPy array to the desired shape
X_train = X_train.reshape(len(X_train), 100, 100, 1)
X_test = X_test.reshape(len(X_test), 100, 100, 1)

y_train=y_train.values
y_test=y_test.values
y_train=y_train.reshape(len(y_train),1)
y_test=y_test.reshape(len(y_test),1)

X_train=X_train/255.0
X_test=X_test/255.0

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7257, 100, 100, 1)
(1815, 100, 100, 1)
(7257, 1)
(1815, 1)


Fifth Step: Define the Models

In [36]:
#CNN Model
cnn_model_1 = Sequential()

# Convolutional layers with Batch Normalization
cnn_model_1.add(Conv2D(kernel_size=(3, 3), filters=64, input_shape=(100, 100, 1), activation='relu'))
cnn_model_1.add(BatchNormalization())
cnn_model_1.add(MaxPooling2D(pool_size=(2, 2)))

cnn_model_1.add(Conv2D(kernel_size=(3, 3), filters=128, activation='relu'))
cnn_model_1.add(BatchNormalization())
cnn_model_1.add(MaxPooling2D(pool_size=(2, 2)))

cnn_model_1.add(Conv2D(kernel_size=(3, 3), filters=256, activation='relu'))
cnn_model_1.add(BatchNormalization())
cnn_model_1.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout layer
cnn_model_1.add(Dropout(0.5))

# Global average pooling layer
cnn_model_1.add(GlobalAveragePooling2D())

# Output layer
cnn_model_1.add(Dense(27, activation='softmax'))

cnn_model_1.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        640       
                                                                 
 batch_normalization (Batch  (None, 98, 98, 64)        256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 batch_normalization_1 (Bat  (None, 47, 47, 128)       512       
 chNormalization)                                                
                                                      

In [38]:
#Compile the model 
cnn_model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the model for 100 epochs
H = cnn_model_1.fit(X_train, y_train, epochs=2)

Epoch 1/2


227/227 [==============================] - 96s 411ms/step - loss: 2.5612 - accuracy: 0.3003
Epoch 2/2
227/227 [==============================] - 88s 386ms/step - loss: 1.6985 - accuracy: 0.5603


In [39]:
print("Evaluate on test data")
results = cnn_model_1.evaluate(X_test, y_test)
print("test loss, test acc:", results)

Evaluate on test data
57/57 [==============================] - 7s 120ms/step - loss: 3.6200 - accuracy: 0.1471
test loss, test acc: [3.6199800968170166, 0.14710743725299835]


SVM

In [22]:
#flatten all the images and store their pixel values
def process_image(img):
    try:
        img_resized = img.resize((100, 100), Image.LANCZOS)
        img_array = np.array(img_resized.convert('L')).flatten()
        return img_array
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

#Create an empty dataframe
image_df = pd.DataFrame()

#Iterate through images in augmented_df
for index, row in augmented_df.iterrows():
    try:
        img_array = process_image(row['Augmented_Image']) 
        if img_array is not None:
            new_row = pd.DataFrame([img_array], columns=[f'pixel_{i}' for i in range(len(img_array))])
            image_df = pd.concat([image_df, new_row], ignore_index=True)
    except Exception as e:
        #Handle errors
        print(f"Error processing image at index {index}: {e}")

print(image_df)

KeyboardInterrupt: 

In [ ]:
#image_array = image_df.values

features=image_df.iloc[:,0:51529]
labels=image_df['Label coded']

X_train_svm, y_train_svm, X_test_svm, y_test_svm=train_test_split(features, labels, test_size=0.2, random_state=42)
X_train_svm=X_train_svm/255.0
X_test_svm=X_test_svm/255.0
#Now we have our training and test sets
print("Training set shape - Features:", X_train_svm.shape, "Labels:", y_train_svm.shape)
print("Testing set shape - Features:", X_test_svm.shape, "Labels:", y_test_svm.shape)

In [14]:
#SVM
from sklearn import svm 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

# Defining the parameters grid for GridSearchCV 
param_grid={'C':[0.1,1,10,100], 
            'gamma':[0.0001,0.001,0.1,1], 
            'kernel':['rbf','poly']} 
  
# Creating a support vector classifier 
svc=svm.SVC(probability=True) 
  
# Creating a model using GridSearchCV with the parameters grid 
model=GridSearchCV(svc,param_grid)

In [20]:
# Training the model using the training data
#NOTE: we need a flattened array here
model.fit(X_train_svm,y_train_svm)

ValueError: Found input variables with inconsistent numbers of samples: [72570000, 7257]

In [ ]:
# Testing the model using the testing data 
y_pred = model.predict(X_test_svm) 
  
# Calculating the accuracy of the model 
accuracy = accuracy_score(y_pred, y_test_svm) 
  
# Print the accuracy of the model 
print(f"The model is {accuracy*100}% accurate")

In [ ]:
print(classification_report(y_test_svm, y_pred, target_names=['XXX']))